In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import random
import cv2
import math
import shutil
from skimage import io
import os
from moviepy.editor import ImageSequenceClip

import torch
from torchvision import transforms

from tqdm.auto import tqdm
from dataloading.nvidia import NvidiaCropWide, Normalize, NvidiaDataset, NvidiaValidationDataset
from trainer import Trainer

%load_ext autoreload
%autoreload 2

In [2]:
import math
import cv2
import shutil
from skimage import io
import os
from moviepy.editor import ImageSequenceClip


def draw_steering_angle(frame, steering_angle, steering_wheel_radius, steering_position, size, color):
    steering_angle_rad = math.radians(steering_angle)
    x = steering_wheel_radius * np.cos(np.pi / 2 + steering_angle_rad)
    y = steering_wheel_radius * np.sin(np.pi / 2 + steering_angle_rad)
    cv2.circle(frame, (steering_position[0] + int(x), steering_position[1] - int(y)), size, color, thickness=-1)
    
def draw_frames(dataset, predicted_angles, predicted_speed, temp_frames_folder):
    
    for frame_index, data in tqdm(enumerate(dataset), total=len(dataset)):
        
        frame = data["image"].permute(1, 2, 0).cpu().numpy()
        true_angle = math.degrees(data["steering_angle"])
        pred_angle = math.degrees(predicted_angles[frame_index])        
        true_speed = data["vehicle_speed"] * 3.6
        pred_speed = predicted_speed[frame_index] * 3.6
        
        position_x = data["position_x"]
        position_y = data["position_y"]
        yaw = math.degrees(data["yaw"])
        turn_signal = int(data["turn_signal"])
        
        cv2.putText(frame, 'True: {:.2f} deg, {:.2f} km/h'.format(true_angle, true_speed), (10, 1150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, 'Pred: {:.2f} deg, {:.2f} km/h'.format(pred_angle, pred_speed), (10, 1200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2,
                    cv2.LINE_AA)
        
        cv2.putText(frame, 'frame: {}'.format(frame_index), (10, 900), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, 'x: {:.2f}'.format(position_x), (10, 950), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, 'y: {:.2f}'.format(position_y), (10, 1000), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, 'yaw: {:.2f}'.format(yaw), (10, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        
        turn_signal_map = {
            1: "straight",
            2: "left",
            0: "right"
        }
        cv2.putText(frame, 'turn signal: {}'.format(turn_signal_map.get(turn_signal, "unknown")), (10, 1100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
    
        radius = 200
        steering_pos = (960, 1200)
        cv2.circle(frame, steering_pos, radius, (255, 255, 255), 7)
        
        cv2.rectangle(frame, (905, 1200), (955, 1200-int(3*true_speed)), (0,255,0), cv2.FILLED)
        cv2.rectangle(frame, (965, 1200), (1015, 1200-int(3*pred_speed)), (255, 0,0), cv2.FILLED)
        

        draw_steering_angle(frame, true_angle, radius, steering_pos, 13, (0, 255, 0))
        draw_steering_angle(frame, pred_angle, radius, steering_pos, 9, (255, 0, 0))
            
        io.imsave(f"{temp_frames_folder}/{frame_index + 1:05}.jpg", frame)
        

def convert_frames_to_video(frames_folder, output_video_path, fps=25):
    output_folder = Path(os.path.split(output_video_path)[:-1][0])
    output_folder.mkdir(parents=True, exist_ok=True)

    p = Path(frames_folder).glob('**/*.jpg')
    image_list = sorted([str(x) for x in p if x.is_file()])

    print("Creating video {}, FPS={}".format(frames_folder, fps))
    clip = ImageSequenceClip(image_list, fps=fps)
    clip.write_videofile(output_video_path)

In [3]:
root_path = Path("/media/romet/data2/datasets/rally-estonia/dataset")
valid_paths = [root_path / "2021-09-30-15-56-59_e2e_ss14_attempt_2"]
validset = NvidiaDataset(valid_paths)
#validset.frames = validset.frames[2000:3000]

/media/romet/data2/datasets/rally-estonia/dataset/2021-09-30-15-56-59_e2e_ss14_attempt_2: 66899


In [4]:
validset.frames.turn_signal.unique()

array([1.        , 0.08925018, 0.        , 0.78510088, 1.84673658,
       2.        , 1.69997956, 0.52925068, 0.02554883, 0.99714301,
       0.10252545, 0.21925966, 0.32426083, 0.16574102, 0.2737292 ,
       0.57079133, 1.85545705, 1.60407426])

In [5]:
#validset = NvidiaValidationDataset(root_path)
tr = transforms.Compose([NvidiaCropWide(), Normalize()])
validloader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=False,
                                         num_workers=16, pin_memory=True, persistent_workers=True)
print(len(validset.frames))

66899


In [6]:
trainer = Trainer("")
torch_model = trainer.load_model(f"models/20211019152624_autumn-v1/best.pt")
torch_model.eval()

validset_tr = NvidiaDataset(valid_paths, tr)
#validset_tr.frames = validset_tr.frames[2000:3000]
validloader_tr = torch.utils.data.DataLoader(validset_tr, batch_size=64, shuffle=False,
                                         num_workers=16, pin_memory=True, persistent_workers=True)
steering_predictions = trainer.predict(torch_model, validloader_tr)

/media/romet/data2/datasets/rally-estonia/dataset/2021-09-30-15-56-59_e2e_ss14_attempt_2: 66899


  0%|          | 0/1046 [00:00<?, ?it/s]

In [7]:
validset_tr.frames

,index,front_wide_filename,autonomous,left_filename,right_filename,steering_angle,vehicle_speed,turn_signal,position_x,position_y,position_z,roll,pitch,yaw,image_path
0,2021-09-30 12:57:00.444675922,front_wide/1633006620444675922.jpg,False,left/1633006620444675922.jpg,right/1633006620444675922.jpg,0.893,0.0,1.0,7218.209251,-16505.883154,159.304113,-0.026213,-0.005978,-1.851631,/media/romet/data2/datasets/rally-estonia/data...
1,2021-09-30 12:57:00.477990866,front_wide/1633006620477990866.jpg,False,left/1633006620477990866.jpg,right/1633006620477990866.jpg,0.893,0.0,1.0,7218.209277,-16505.883138,159.304102,-0.026213,-0.005993,-1.851632,/media/romet/data2/datasets/rally-estonia/data...
2,2021-09-30 12:57:00.511337042,front_wide/1633006620511337042.jpg,False,left/1633006620511337042.jpg,right/1633006620511337042.jpg,0.893,0.0,1.0,7218.209284,-16505.883130,159.304091,-0.026220,-0.005983,-1.851626,/media/romet/data2/datasets/rally-estonia/data...
3,2021-09-30 12:57:00.544673920,front_wide/1633006620544673920.jpg,False,left/1633006620544673920.jpg,right/1633006620544673920.jpg,0.893,0.0,1.0,7218.209296,-16505.883125,159.304079,-0.026205,-0.005987,-1.851644,/media/romet/data2/datasets/rally-estonia/data...
4,2021-09-30 12:57:00.578004837,front_wide/1633006620578004837.jpg,False,left/1633006620578004837.jpg,right/1633006620578004837.jpg,0.893,0.0,1.0,7218.209289,-16505.883127,159.304068,-0.026204,-0.005974,-1.851629,/media/romet/data2/datasets/rally-estonia/data...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66894,2021-09-30 13:34:10.277755022,front_wide/1633008850277755022.jpg,False,left/1633008850277755022.jpg,right/1633008850277755022.jpg,0.613,0.0,1.0,4942.964297,-15144.341280,148.255971,-0.001995,-0.005947,0.778623,/media/romet/data2/datasets/rally-estonia/data...
66895,2021-09-30 13:34:10.311098814,front_wide/1633008850311098814.jpg,False,left/1633008850311098814.jpg,right/1633008850311098814.jpg,0.613,0.0,1.0,4942.964331,-15144.341276,148.255897,-0.002010,-0.005951,0.778619,/media/romet/data2/datasets/rally-estonia/data...
66896,2021-09-30 13:34:10.344434977,front_wide/1633008850344434977.jpg,False,NaN,NaN,0.613,0.0,1.0,4942.964364,-15144.341268,148.255833,-0.002002,-0.005943,0.778628,/media/romet/data2/datasets/rally-estonia/data...
66897,2021-09-30 13:34:10.377763987,front_wide/1633008850377763987.jpg,False,left/1633008850377763987.jpg,right/1633008850377763987.jpg,0.613,0.0,1.0,4942.964385,-15144.341261,148.255770,-0.002020,-0.005941,0.778621,/media/romet/data2/datasets/rally-estonia/data...


In [8]:
from velocity_model.velocity_model import VelocityModel
velocity_model = VelocityModel(positions_parquet='velocity_model/positions.parquet')

frames_df = validset_tr.frames

x = frames_df["position_x"] + np.random.normal(0, 0.1, len(frames_df))
y = frames_df["position_y"] + np.random.normal(0, 0.1, len(frames_df))
yaw = frames_df["yaw"] + np.random.normal(0, 0.2, len(frames_df))

result_df = pd.DataFrame(data={'x': x, 'y': y, 'yaw': yaw})
speed_predictions = result_df.apply(lambda x: velocity_model.find_speed_for_position(x['x'], x['y'], x['yaw'])[0], axis=1)
speed_predictions = speed_predictions.to_numpy()

In [9]:
temp_frames_folder = Path("./temp_frames")
shutil.rmtree(temp_frames_folder, ignore_errors=True)
temp_frames_folder.mkdir()

draw_frames(validset, steering_predictions, speed_predictions, temp_frames_folder)
output_video = "output/test.mp4"
convert_frames_to_video(temp_frames_folder, output_video, fps=30.0)

shutil.rmtree(temp_frames_folder, ignore_errors=True)

  0%|          | 0/66899 [00:00<?, ?it/s]

Creating video temp_frames, FPS=30.0
Moviepy - Building video output/test.mp4.
Moviepy - Writing video output/test.mp4



Moviepy - Done !
Moviepy - video ready output/test.mp4


In [10]:
from IPython.display import HTML


HTML(f"""
<video width="640" height="480" controls>
  <source src="{output_video}" type="video/mp4">
</video>
""")